<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/sparsity_selectivity_2_HL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

root_dir = './'
torchvision.datasets.MNIST(root=root_dir,download=True)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
torch.manual_seed(1234)
np.random.seed(1234)

In [5]:
mnist_trainset = torchvision.datasets.MNIST(root=root_dir, train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root=root_dir, 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [6]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 256)
        self.linear_3 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()
        self.sigmoid23  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        x = self.linear_2(x)
        x = self.sigmoid23(x)
        pred = self.linear_3(x)
        return pred
# ************* modify this section for later use *************

In [7]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [8]:
def sparsity_calculator(final_spareness):
    sparseness_list = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return sparseness_list

In [9]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]
    
    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [10]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12, _hidden_layer_each_neuron_23):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)
    avg_selectivity23, std_selectivity23 = selectivity(_hidden_layer_each_neuron_23)

    final_selectivity_avg = (avg_selectivity12 + avg_selectivity23) / 2 
    final_selecvitity_std = (std_selectivity12 + std_selectivity23) / 2 

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [11]:
def model_factory(optimizer_name):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    my_model.sigmoid23.register_forward_hook(get_activation(my_model, 's23'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, selectivity_list_avg, selectivity_list_std = selectivity_trainer(optimizer=my_optimizer, model=my_model)
    # ************* modify this section for later use *************
    file_saver = open(f"2HL_selectivity_sparsity_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    # ************* modify this section for later use *************
    if optimizer_name == 'Adadelta':
        !cp 2HL_selectivity_sparsity_Adadelta.txt /content/drive/MyDrive
    
    elif optimizer_name == 'Adagrad':
        !cp 2HL_selectivity_sparsity_Adagrad.txt /content/drive/MyDrive

    elif optimizer_name == 'SGD':
        !cp 2HL_selectivity_sparsity_SGD.txt /content/drive/MyDrive

    elif optimizer_name == 'Adam':
        !cp 2HL_selectivity_sparsity_Adam.txt /content/drive/MyDrive
    # ************* modify this section for later use *************

    else:
        print("ERROR")

In [12]:
no_epochs = 50
def selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    final_spareness_23 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)

        hidden_layer_each_neuron_23 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_23 = np.array(hidden_layer_each_neuron_23)
        # ************* modify this section for later use *************

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        hidden_layer_activation_list_23 = list()
        # ************* modify this section for later use *************

        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])
            hidden_layer_activation_list_23.append(model.layer_activations['s23'])

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])

            for activation, label in zip(model.layer_activations['s23'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_23[i][label].append(activation[i])
    
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12, hidden_layer_each_neuron_23)
        # ************* modify this section for later use *************
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        final_spareness_23.append(hidden_layer_activation_list_23)
        # ***************** sparsity calculation ***************** #

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12 = sparsity_calculator(final_spareness_12)
    sparsity_list23 = sparsity_calculator(final_spareness_23)

    print(sparsity_list12)
    print(sparsity_list23)

    average_sparsity = list()
    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12[i].item() + sparsity_list23[i].item()) / 2 )

    # ***************** sparsity calculation ***************** #

    print("average_sparsity:", average_sparsity)

    return test_acc, average_sparsity, selectivity_avg_list, selectivity_std_list

# Adadelta

In [13]:
model_factory('Adadelta')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 0.66955220, Test Loss: 0.29091561, Test Accuracy: 0.91560000

Epoch: 2/50, Train Loss: 0.23864885, Test Loss: 0.17893777, Test Accuracy: 0.94570000

Epoch: 3/50, Train Loss: 0.16415371, Test Loss: 0.14251488, Test Accuracy: 0.95830000

Epoch: 4/50, Train Loss: 0.12487799, Test Loss: 0.10738104, Test Accuracy: 0.96570000

Epoch: 5/50, Train Loss: 0.09811718, Test Loss: 0.09940851, Test Accuracy: 0.96730000

Epoch: 6/50, Train Loss: 0.08101228, Test Loss: 0.09228467, Test Accuracy: 0.97040000

Epoch: 7/50, Train Loss: 0.06785859, Test Loss: 0.08866796, Test Accuracy: 0.97140000

Epoch: 8/50, Tra

# AdaGrad

In [14]:
model_factory('Adagrad')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
)
my_optimizer: Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 0.36868044, Test Loss: 0.11248965, Test Accuracy: 0.96570000

Epoch: 2/50, Train Loss: 0.08117606, Test Loss: 0.08445396, Test Accuracy: 0.97210000

Epoch: 3/50, Train Loss: 0.05059796, Test Loss: 0.07754592, Test Accuracy: 0.97690000

Epoch: 4/50, Train Loss: 0.03388871, Test Loss: 0.07117682, Test Accuracy: 0.97760000

Epoch: 5/50, Train Loss: 0.02313452, Test Loss: 0.07472019, Test Accuracy: 0.97740000

Epoch: 6/50, Train Loss: 0.01545731, Test Loss: 0.07222190, Test Accuracy: 0.98050000

Epoch: 7/50, Train Loss: 0.01061224, Test Loss: 0.07550140, Test Acc

# SGD


In [15]:
model_factory('SGD')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
)
my_optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 1.69265915, Test Loss: 0.75299979, Test Accuracy: 0.76860000

Epoch: 2/50, Train Loss: 0.54849599, Test Loss: 0.42523081, Test Accuracy: 0.87280000

Epoch: 3/50, Train Loss: 0.38982520, Test Loss: 0.34583928, Test Accuracy: 0.89970000

Epoch: 4/50, Train Loss: 0.34312607, Test Loss: 0.31302324, Test Accuracy: 0.90980000

Epoch: 5/50, Train Loss: 0.31391889, Test Loss: 0.29038461, Test Accuracy: 0.91460000

Epoch: 6/50, Train Loss: 0.28898875, Test Loss: 0.27226443, Test Accuracy: 0.92060000

Epoch: 7/50, Train Loss: 0.26564695, Test Loss: 0.24730698, Test Accuracy: 0.928000

# Adam

In [16]:
model_factory('Adam')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 0.46062646, Test Loss: 0.23019592, Test Accuracy: 0.93020000

Epoch: 2/50, Train Loss: 0.17090873, Test Loss: 0.13047612, Test Accuracy: 0.95990000

Epoch: 3/50, Train Loss: 0.11648787, Test Loss: 0.11877710, Test Accuracy: 0.96170000

Epoch: 4/50, Train Loss: 0.08605863, Test Loss: 0.09043822, Test Accuracy: 0.97080000

Epoch: 5/50, Train Loss: 0.06640637, Test Loss: 0.09237713, Test Accuracy: 0.97000000

Epoch: 6/50, Train Loss: 0.05035246, Test Loss: 0.07154697, Test Accuracy: 0.97760000

Epoch: 7/50, Train Loss: 0.04009322, Test Loss: 0.07882151, Test Accuracy: 